# Calibrating machine-learning models to increase the prediction accuracy of specific experiments
### Aleš Křenek
#### Sitola, 22.9.2021

## Two-slide quick tour to LC/MS
<img src="lc.jpg" style="width: 15vw;" align="right"/>

- Analyzed sample -- mixture of unknown compounds
- Diluted in some liquid (MeOH) and pushed through *chromatographic column*
- Complex interaction among sample, liquid, and column surface
- Results in separation of compounds in varying *retention time* (RT)

## Two-slide quick tour to LC/MS
<img src="masspec.png" style="width: 25vw;" align="right"/>

- Separated compounds fed to mass spectrometer
- Results in 2D spectrum
- Software signal processing and library search magic
- List of candidate compounds at each point in time
- *Too many false positives*

## RT prediction

### Use for disambiguation
- MS says that the spectrum at time $t$ can be either compound $A$ or $B$
- If we knew expected times for $A$ and $B$, and they were different, we could pick the right one

### Modeling chromatography
- Complex process, difficult to model in traditional way
- Suitable for machine learning
- Reference dataset (known times for a set of compounds) is required


In [ ]:
# just bookkeeping
featf='features.csv'
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
from os import getcwd,chdir
from moldescriptors import get_features
from main import make_preds
datadir='/data'
prefix='/opt/callc/rt/'
modlib=prefix + 'mods_l1/'
models=['bayesianregr','lasso']
featn=[x.strip() for x in open(prefix+"features/selected_features.txt").readlines()]
featn.remove('system')
with open('feats_lib.csv','w'): pass

## Train a fresh model with our data

In [ ]:
from sklearn.linear_model import ARDRegression
from scipy.stats import randint
from scipy.stats import uniform
import random
random.seed(815)
model = ARDRegression()
params = {
        "n_iter" : randint(100,1500),
        "alpha_1" : uniform(1e-10,1e-2),
        "lambda_1" : uniform(1e-10,1e-2),
        "threshold_lambda" : randint(1,10000),
    }

## Formulae and features
- SMILES (Simplified Molecular Input Line Entry System) format: CN1CCCC1C2=CN=CC=C2
<img src="nikotine.png" style="width: 20vw;"/>
- Standardized ways to calculate chemical *features* from the formula

In [ ]:
get_features(infile_name='nikotine.tsv',outfile_name=featf,id_index=0,mol_index=1,time_index=2)
pd.read_csv(featf)

### Train the model on our reference dataset

In [ ]:
get_features(infile_name='train_positive.tsv',outfile_name=featf,id_index=3,mol_index=0,time_index=2)
features = pd.read_csv(featf)[featn]
features.shape

In [ ]:
from random import shuffle
testsize=int(len(features.index)*.2)
index=list(range(len(features.index)))
shuffle(index)
test=features.iloc[index[:testsize]]
train=features.iloc[index[testsize:]]

In [ ]:
from sklearn.model_selection import KFold
from trainl1 import train_model_l1
cv = list(KFold(n_splits=10,shuffle=True).split(train.index))
model,train_preds = train_model_l1(train.drop(["time","IDENTIFIER","system"],axis=1, errors="ignore"),
                                             train["time"],params,model,
                                             cv = cv,n_params=20,
                                             n_jobs=4)

In [ ]:
def rtscatter(rt,mod):
    plt.figure(figsize=(8,6))
    plt.plot([200,1200],[200,1200],color='grey')
    plt.scatter(rt,mod)
    plt.show()

In [ ]:
rtscatter(train['time'],train_preds)

### Apply on independent test set

In [ ]:
test_preds = model.predict(test.drop(["time","IDENTIFIER"],axis=1))

In [ ]:
rtscatter(test['time'],test_preds)

### How good is the model?
* Plot the graph and assess intuitively -- "chi by eye"
* Or _Coeffitient of determination_ $R^2$
* Intuitively: How much variance in the data the model explains?
  * $R^2 = 1$ -- the model is perfect
  * $R^2 = 0$ -- the model is not better than expecting average value of $y$
  * $R^2 < 0$ or $R^2 > 1$ -- the model is unusable at all

In [ ]:
def r2(measured,model):
    mean = np.average(measured)
    yres = measured-mean
    modres = model-measured
    ss_tot = np.sum(yres*yres)
    ss_res = np.sum(modres*modres)
    return 1. - ss_res/ss_tot

In [ ]:
r2(np.array(test['time']),test_preds)

- Not entirely bad (the model works, somehow)
- Not overimpressive either
- **Too small reference dataset to build in-house model**

## Apply available models on our data
- Someone else more lucky (more rich) could have measured bigger datasets
- We can try reusing their models

In [ ]:
def apply_model(X,modname):
    modf=modlib + modname + '42.pickle'
    with open(modf,"rb") as m:
        model = pickle.load(m,encoding='latin1')
    preds=model.predict(X)
    return preds

In [ ]:
dataset='MTBLS20'
preds={}
for m in models:
    preds[m] = apply_model(features.drop(['IDENTIFIER','time'],axis=1),modname=dataset+'_'+m)

In [ ]:
bayes_good=(preds['bayesianregr'] < 1200) # get rid of apparent outliers
rtscatter(np.array(features['time'])[bayes_good],preds['bayesianregr'][bayes_good])

In [ ]:
r2(np.array(features['time'])[bayes_good],preds['bayesianregr'][bayes_good])

In [ ]:
lasso_good=(np.abs(preds['lasso']) < 1000)
rtscatter(np.array(features['time'])[lasso_good],preds['lasso'][lasso_good])

In [ ]:
r2(np.array(features['time'])[lasso_good],preds['lasso'][lasso_good])

- Models built on other lab datasets are unusable in general
- **Every chromatographic column is unique**

## CALLC main ideas
- *Callibrated All Liquid Chromatography* 
- Gather $N$ existing datasets: known compounds and RT for a specific laboratory setup
- Train $M$ ML models on them independently, yielding $M\times N$ models altogether
- Measure a representative small *callibration* dataset in your lab
- Train the same ML models on the callibration dataset
- Apply all $M\times (N+1)$ models callibration dataset
- Find the best *GAM (generalized additive model)* curves to transform the models outputs
- Pick the right set of models and their linear combination with *elastic net*

## Employ the big beast

In [ ]:
chdir(datadir)
train.to_csv('reference.csv')
test.to_csv('test.csv')
chdir(prefix)
make_preds(reference_infile=datadir + '/reference.csv',pred_infile=datadir + '/test.csv',
           outfile=datadir+'/test_preds')

In [ ]:
chdir(datadir)
big_pred=pd.read_csv('test_preds.csv')

In [ ]:
plt.figure(figsize=(10,8))
plt.plot([200,1000],[200,1000],label='diagonal',color='grey')
plt.scatter(test['time'],big_pred['predictions'],label='full model')
plt.scatter(test['time'],test_preds,label='in house only')
plt.legend()
plt.show()

In [ ]:
r2(np.array(test['time']),np.array(test_preds['predictions']))

In [ ]:
r2(np.array(test['time']),big_pred['predictions'])

# General conclusions
- Not specific problem to LC/MS and RT prediction only
- Complex behaviour, difficult to model, suitable for machine learning
- Not enough training data available in-house
- More datasets elsewhere, not directly transferable, but still somehow similar
- This work describes how to callibrate and mix "the other" models